In [1]:
##read Data Base
from pymongo import MongoClient
from bson.objectid import ObjectId
uri = "mongodb://localhost:27017/database" #mongodb://<user_name>:<user_password>@ds<xxxxxx>.mlab.com:<xxxxx>/<database_name>
conn =  MongoClient(host='localhost', port=27017, connect=False)
db = conn.PaperData
Spectrogram_Collection  =  db.get_collection('AudioSpectrogram_100thTag')
#LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')

In [2]:
#Audio Para
_mel_scale = 96
_time_len = 1292
_channels = 1

In [3]:
_num_Tags = 50
_epchos = 200
_stopEpcho = 20
_batchSize = 32

In [4]:
import json 
import codecs
import numpy as np
def read_json(filename):
    with codecs.open(filename,'r',encoding = 'utf8') as infile:
        return np.array(json.load(infile))
    
x_train = read_json('x_train_'+str(_num_Tags)+'d_sub.json')
x_test = read_json('x_test_'+str(_num_Tags)+'d_sub.json')
y_train = read_json('y_train_'+str(_num_Tags)+'d_sub.json')
y_test = read_json('y_test_'+str(_num_Tags)+'d_sub.json')

In [5]:
from keras.layers import *
from keras.models import Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical

Using TensorFlow backend.


In [6]:
Spectrogram_Input = Input(name= 'AudioInput',shape=(_mel_scale, _time_len, _channels)) #(None(1), 96, 1292, 1)
CL_1 = Conv2D(128, kernel_size=(3, 3),padding='same', name = "CL_1")(Spectrogram_Input) #(None, 96, 1292, 128) 
BN_1 = BatchNormalization()(CL_1)
AL_1 = ELU()(BN_1)
MP_1 = MaxPool2D(pool_size= (2,4), name = "MPL_1")(AL_1) #(None, 48, 323, 128)
DP_1 = Dropout(0.5)(MP_1)

CL_2 = Conv2D(384, kernel_size=(3, 3),padding='same', name = "CL_2")(DP_1) # (None, 48, 323, 384) 
BN_2 = BatchNormalization()(CL_2)
AL_2 = ELU()(BN_2)
MP_2 = MaxPool2D(pool_size= (4,5), name = "MPL_2")(AL_2) # (None, 12, 64, 384) 
DP_2 = Dropout(0.5)(MP_2)

CL_3 = Conv2D(768, kernel_size=(3, 3),padding='same', name = "CL_3")(DP_2) #(None, 12, 64, 768) 
BN_3 = BatchNormalization()(CL_3)
AL_3 = ELU()(BN_3)
MP_3 = MaxPool2D(pool_size= (3,8), name = "MPL_3")(AL_3) #(None, 4, 8, 768)
DP_3 = Dropout(0.5)(MP_3)

CL_4 = Conv2D(2048, kernel_size=(3, 3),padding='same', name = "CL_4")(DP_3) #(None, 4, 8, 2048) 
BN_4 = BatchNormalization()(CL_4)
AL_4 = ELU()(BN_4)
MP_4 = MaxPool2D(pool_size= (4,8), name = "MPL_4")(AL_4)#(None, 1, 1, 2048) 
DP_4 = Dropout(0.5)(MP_4)

AF = Flatten()(DP_4) # (None, 2048) 
#pseudo-model for sheck summary
Softmax = Dense(_num_Tags, kernel_initializer='normal',activation='sigmoid')(AF)
Audio_model = Model(inputs=Spectrogram_Input, outputs=Softmax)

In [7]:
Audio_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
AudioInput (InputLayer)      (None, 96, 1292, 1)       0         
_________________________________________________________________
CL_1 (Conv2D)                (None, 96, 1292, 128)     1280      
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1292, 128)     512       
_________________________________________________________________
elu_1 (ELU)                  (None, 96, 1292, 128)     0         
_________________________________________________________________
MPL_1 (MaxPooling2D)         (None, 48, 323, 128)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 323, 128)      0         
_________________________________________________________________
CL_2 (Conv2D)                (None, 48, 323, 384)      442752    
__________

In [8]:
from keras.utils import Sequence
import json
import codecs
class AudioDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels,batch_size):
        self.text_filenames, self.labels = text_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.text_filenames)/ (self.batch_size)))

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename_List =  self.text_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        #Text_batch_x = [] 
        Audio_batch_x = []
        for filename in filename_List:  
            Audio_x = np.array(Spectrogram_Collection.find_one({"Filename":filename})['Spectrogram']).reshape((_mel_scale,_time_len,_channels))
            Audio_batch_x.append(Audio_x)
        y_List = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array(Audio_batch_x), np.array(y_List)
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [9]:
ADG_train = AudioDataGenerator(x_train, y_train, _batchSize)
ADG_test = AudioDataGenerator(x_test, y_test, _batchSize)

In [10]:
x,y = ADG_train.getitem(0)

In [11]:
x.shape

(32, 96, 1292, 1)

In [12]:
y.shape

(32, 50)

In [13]:
x.max()

1.0

In [14]:
x.min()

0.0

In [15]:
##This AUC callback is for STL

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC_callback(Callback):
    def __init__(self,y,SDG,log_name):
        self.y = y
        self.SDG = SDG
        self.log_name = log_name
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        Skip_Tag = []
        
        y_pred = self.model.predict_generator(self.SDG) # (#data x 50()) 
        #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
        for i in range(len(y_pred)):
            for j in range(len(y_pred[i])):
                if y_pred[i][j] >= 0.5 : 
                    y_pred[i][j] = 1
                else:
                    y_pred[i][j] = 0
        
        #print(y_pred.shape,self.y.shape)
        
        
        AUC_this_epoch = roc_auc_score(self.y ,y_pred)
        logs[self.log_name] = AUC_this_epoch
        print('%s : %s '% (self.log_name,(str(round(AUC_this_epoch,5)))))
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [16]:
AUC_test_callback =AUC_callback(y_test,ADG_test,'AUC_test')
AUC_train_callback =AUC_callback(y_train,ADG_train,'AUC_train')

In [17]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=_stopEpcho, verbose=1)

In [18]:
import time
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.epoch_time_end = time.time()
        self.times.append(self.epoch_time_end - self.epoch_time_start)
        logs['Timer'] = self.epoch_time_end - self.epoch_time_start
time_callback = TimeHistory()


In [19]:
def multitask_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

In [20]:
import keras_metrics
precision = keras_metrics.precision(label = 1)
recall = keras_metrics.recall(label = 0)

In [21]:
#https://www.quora.com/Why-does-my-convolutional-neural-network-always-produce-the-same-outputs

In [22]:
num_training_samples = len(x_train)
num_validation_samples = len(x_test)

In [24]:
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(Audio_model, gpus=2)
parallel_model.compile(loss="binary_crossentropy",
                      optimizer='adam',
                      metrics=['acc'])
History = parallel_model.fit_generator(
                    generator= ADG_train,
                    steps_per_epoch=(num_training_samples // _batchSize),
                    epochs= 200,
                    verbose=1,
                    validation_data= ADG_test,
                    validation_steps=(num_validation_samples // _batchSize),
                    workers=12, use_multiprocessing=True,
                    callbacks = [early_stopping,time_callback]
                    )

Epoch 1/200


/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo

652/653 [============================>.] - ETA: 0s - loss: 0.5492 - acc: 0.8100

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


653/653 [==============================] - 212s 325ms/step - loss: 0.5491 - acc: 0.8100 - val_loss: 0.4289 - val_acc: 0.8447
Epoch 2/200
  1/653 [..............................] - ETA: 6:49 - loss: 0.4599 - acc: 0.8262

...

653/653 [==============================] - 201s 308ms/step - loss: 0.2539 - acc: 0.8940 - val_loss: 0.3448 - val_acc: 0.8620
Epoch 48/200
 47/653 [=>............................] - ETA: 2:36 - loss: 0.2534 - acc: 0.8950

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo

652/653 [============================>.] - ETA: 0s - loss: 0.2508 - acc: 0.8957

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo

653/653 [==============================] - 203s 311ms/step - loss: 0.2508 - acc: 0.8957 - val_loss: 0.3466 - val_acc: 0.8625
Epoch 00048: early stopping


In [32]:
from keras.models import model_from_json
json_string = parallel_model.to_json()
with codecs.open('Audio_STL.json','w', encoding = 'utf8') as outfile:
    json.dump(json_string,outfile)

In [34]:
with codecs.open('Audio_STL_History.json','w', encoding = 'utf8') as outfile:
    json.dump(History.history,outfile)

In [26]:
small_ADG =AudioDataGenerator(x_test[:_batchSize], y_test[:_batchSize], _batchSize)

In [27]:
sult = parallel_model.predict_generator(small_ADG)

In [28]:
sult.shape

(32, 50)

In [31]:
sult[11]

array([1.85913056e-01, 1.78026259e-01, 4.31507170e-01, 7.44856417e-01,
       4.12452937e-04, 1.00411594e-01, 3.68130356e-02, 8.64118189e-02,
       8.25427356e-04, 8.99636596e-02, 2.10930128e-02, 1.36609888e-03,
       3.40320081e-01, 6.58634007e-01, 1.16355797e-04, 6.03099279e-02,
       7.89061934e-02, 9.11619514e-03, 1.31636520e-03, 7.99881585e-04,
       7.34731779e-02, 4.41600800e-01, 1.00960358e-04, 9.46183443e-01,
       1.30719738e-03, 1.55229755e-02, 1.10087777e-02, 1.61600098e-01,
       2.62221061e-02, 5.18581737e-03, 8.95796635e-04, 5.97145036e-03,
       4.21919897e-02, 3.78001742e-02, 1.94200387e-04, 2.33035721e-02,
       3.88431214e-02, 1.00956224e-02, 9.54848886e-01, 2.94251018e-03,
       2.40020826e-02, 5.93081824e-02, 3.30718070e-01, 8.32138630e-06,
       1.98210659e-03, 2.59802141e-03, 2.92398464e-02, 2.18054429e-01,
       1.72544416e-04, 9.51953046e-03], dtype=float32)

In [30]:
sult[31]

array([0.16197494, 0.07874324, 0.12413406, 0.37253064, 0.1583796 ,
       0.15321854, 0.1400041 , 0.0473436 , 0.02602292, 0.18592478,
       0.03368156, 0.00459933, 0.053838  , 0.02243373, 0.07508249,
       0.02276694, 0.01705349, 0.11199069, 0.00453027, 0.00259375,
       0.16547252, 0.05024329, 0.08040342, 0.20501226, 0.00575106,
       0.01815371, 0.04375155, 0.02206996, 0.24994038, 0.07543863,
       0.03413051, 0.00240376, 0.31120732, 0.00226821, 0.01088453,
       0.01826786, 0.06691474, 0.12518749, 0.00817162, 0.3330595 ,
       0.04132319, 0.02851249, 0.00227338, 0.01362648, 0.00467118,
       0.0177607 , 0.00192396, 0.00250571, 0.03845548, 0.04913447],
      dtype=float32)